In [3]:
import os
import mlflow
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
import pandas as pd

%matplotlib inline

#alt.renderers.enable("html")
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
tracking_uri = os.getenv("TRACKING_URI", default="http://localhost:5000")
mlflow.set_tracking_uri(tracking_uri)

## Experiment Random Parameter-Search TabNet

- Single random seed for model training for all models
- Single random seed for random split for all models
- Per run/model calculate atomic attribution using various methods (e.g. tabnet, integrated gradients, saliency, etc.)
- Single target objective (first) using BCE-loss
- Hyperparameter search using random sampler

In [5]:
experiment_name = "herg_tn_opt2609_3"
exp = mlflow.get_experiment_by_name(experiment_name)

runs = mlflow.search_runs(experiment_ids=exp.experiment_id)

In [ ]:
#runs.iloc[1:].describe().T

In [ ]:
#runs.iloc[1:].corr()

### Search space

In [6]:
search_space = [c for c in runs.columns if "search_space/" in c]
for r in runs[search_space].iloc[0].head(n=99):
    print(r)

{'name': 'momentum', 'type': 'choice', 'values': [0.2, 0.1, 0.05, 0.02]}
{'name': 'virtual_batch_size', 'type': 'choice', 'values': [16, 32, 64, 128]}
{'name': 'nr_steps', 'type': 'choice', 'values': [1, 2, 3, 4, 5]}
{'name': 'lambda_sparse', 'type': 'choice', 'values': [0.0, 1e-06, 0.0001, 0.01]}
{'name': 'decay_rate', 'type': 'choice', 'values': [0.4, 0.8, 0.9, 0.95]}
{'name': 'gamma', 'type': 'choice', 'values': [1.0, 1.2, 1.5]}
{'name': 'lr', 'type': 'choice', 'values': [0.03, 0.02, 0.01]}
{'name': 'decay_step', 'type': 'choice', 'values': [50, 200, 800]}
{'name': 'decision_size', 'type': 'choice', 'values': [8, 16, 24, 32, 64]}


### Other + default parameters

In [7]:
params = [c for c in runs.columns if "search_space" not in c and "params.args" in c]
for p, v in zip(params, runs[params].iloc[0].head(n=99)):
    print(f"{p.replace('params.args/', '')}: {v}")

gradient_clip_val: 1.0
track_metrics: ...value too long for mlflow - not inserted
attribution_kwargs: ...value too long for mlflow - not inserted
scheduler: exponential_decay
nr_shared_layers: 2
split_seed: 381515060
tracking_uri: http://localhost:5000
trials: 30
checkpoint_minimize: True
scheduler_params: {'decay_step': 10, 'decay_rate': 0.95}
sampler_name: random
optimizer: adam
normalize_input: True
split_size: (0.6, 0.2, 0.2)
stochastic_weight_avg: False
run_name: vanilla_random
standardize: False
max_steps: 1000
attentive_type: sparsemax
nr_steps: 4
pruner_name: None
feature_size: 128
alpha: 2.0
minimize: False
patience: 10
nr_layers: 2
gamma: 1.5
num_workers: 8
batch_size: 512
momentum: 0.01
checkpoint_objective: val/loss
lambda_sparse: 0.0
featurizer_kwargs: {'fold': 1024, 'radius': 3, 'return_count': True, 'use_chirality': True, 'use_features': True}
objective_name: val/AUROC
log_sparsity: True
relaxation_type: gamma_fixed
decision_size: 64
cache_dir: ../../../data/herg/
use_la

### Best parameters
- Found within this scenario

In [8]:
params = [c for c in runs.columns if c and "params.best" in c]
for p, v in zip(params, runs[params].iloc[0].head(n=99)):
    print(f"{p.replace('params.args/', '')}: {v}")

params.best/decay_step: 200
params.best/lambda_sparse: 0.0
params.best/nr_steps: 1
params.best/momentum: 0.05
params.best/gamma: 1.2
params.best/decision_size: 64
params.best/virtual_batch_size: 32
params.best/decay_rate: 0.8
params.best/lr: 0.03


### Metrics
- Include metrics for atomic attribution/weights
    - Values were calculated using active hergophores on all rows for which herg activity was predicted as negative
    - Threshold was calculated per individual model using threshold moving

In [15]:
metrics_names = [
    "metrics.test/sparsity_mask",
    "metrics.test/AUROC",
    "metrics.test/loss",
    "metrics.test/mean/avg_score_pred_inactive/tabnet"
    "metrics.test/mean/avg_score_pred_inactive/integrated_gradients",
    "metrics.test/mean/avg_score_pred_inactive/saliency",
]
metrics_columns = [c for c in runs.columns if any(c in m for m in metrics_names)]
metrics = runs[metrics_columns].iloc[1:]

metrics.columns = [c.split("/")[-1] for c in metrics.columns]
metrics.describe().T

,count,mean,std,min,25%,50%,75%,max
loss,30.0,0.561281,0.062008,0.472404,0.522895,0.541905,0.593493,0.677291
tabnet,30.0,0.480297,0.046869,0.409666,0.454300,0.473094,0.496176,0.637701
sparsity_mask,30.0,0.996152,0.002239,0.990601,0.995768,0.996338,0.997805,0.998786
saliency,30.0,0.537360,0.028013,0.497771,0.519249,0.533212,0.548711,0.629568
AUROC,30.0,0.780215,0.086470,0.575788,0.747562,0.824432,0.838485,0.864240
integrated_gradients,30.0,0.612627,0.060814,0.499851,0.573504,0.609824,0.651316,0.743746


#### Correlation

- Correlation between various metrics

In [16]:
metrics.corr(method="pearson")

,loss,tabnet,sparsity_mask,saliency,AUROC,integrated_gradients
loss,1.000000,-0.341033,-0.710007,-0.178331,-0.976440,-0.671885
tabnet,-0.341033,1.000000,0.310793,0.001259,0.196554,0.272255
sparsity_mask,-0.710007,0.310793,1.000000,0.104873,0.687637,0.642951
saliency,-0.178331,0.001259,0.104873,1.000000,0.160894,0.303677
AUROC,-0.976440,0.196554,0.687637,0.160894,1.000000,0.636398
integrated_gradients,-0.671885,0.272255,0.642951,0.303677,0.636398,1.000000


In [12]:
x = "AUROC"
y = "integrated_gradients"

chart = alt.Chart(metrics).mark_point().encode(
    x=alt.X(x, scale=alt.Scale(zero=False)),
    y=alt.Y(y, scale=alt.Scale(zero=False))
)

chart = chart + chart.transform_regression(x, y).mark_line()
chart.show()

Displaying chart at http://localhost:22980/


In [13]:
x = "AUROC"
y = "tabnet"

chart = alt.Chart(metrics).mark_point().encode(
    x=alt.X(x, scale=alt.Scale(zero=False)),
    y=alt.Y(y, scale=alt.Scale(zero=False))
)

chart = chart + chart.transform_regression(x, y).mark_line()
chart.show()

Displaying chart at http://localhost:22980/


In [17]:
x = "sparsity_mask"
y = "tabnet"

chart = alt.Chart(metrics).mark_point().encode(
    x=alt.X(x, scale=alt.Scale(zero=False)),
    y=alt.Y(y, scale=alt.Scale(zero=False))
)

chart = chart + chart.transform_regression(x, y).mark_line()
chart.show()

Displaying chart at http://localhost:22980/


In [18]:
x = "sparsity_mask"
y = "AUROC"

chart = alt.Chart(metrics).mark_point().encode(
    x=alt.X(x, scale=alt.Scale(zero=False)),
    y=alt.Y(y, scale=alt.Scale(zero=False))
)

chart = chart + chart.transform_regression(x, y).mark_line()
chart.show()

Displaying chart at http://localhost:22980/


KeyboardInterrupt: 